# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from glob import glob

from pyopia.classify import Classify
import exampledata
import pyopia.io
from pyopia.pipeline import Pipeline
from pyopia.instrument.silcam import SilCamLoad, ImagePrep
import pyopia.process

In [3]:
filename = exampledata.get_example_silc_image()

files = glob('raw_data/*.silc')

In [4]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/test'
model_path = exampledata.get_example_model()
threshold = 0.85

steps = {'classifier': Classify(model_path=model_path),
         'load': SilCamLoad(),
         'imageprep': ImagePrep(),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps, initial_steps=['classifier'])

Initialising pipeline
  Running <pyopia.classify.Classify object at 0x0000016128F41160>
Pipeline ready with these data:  ['cl']


In [5]:
for filename in files[:2]:
    stats = processing_pipeline.run(filename)

calling:  <class 'pyopia.instrument.silcam.SilCamLoad'>  with:  ['cl', 'filename', 'steps_string']
calling:  <class 'pyopia.instrument.silcam.ImagePrep'>  with:  ['cl', 'filename', 'steps_string', 'timestamp', 'img']
calling:  <class 'pyopia.process.Segment'>  with:  ['cl', 'filename', 'steps_string', 'timestamp', 'img', 'imc']
segment
clean
calling:  <class 'pyopia.process.CalculateStats'>  with:  ['cl', 'filename', 'steps_string', 'timestamp', 'img', 'imc', 'imbw']
statextract_light
21.7% saturation
measure
  870 particles found
WARNING. exportparticles temporarily modified for 2-d images without color!
EXTRACTING 870 IMAGES from 870
calling:  <class 'pyopia.io.StatsH5'>  with:  ['cl', 'filename', 'steps_string', 'timestamp', 'img', 'imc', 'imbw', 'stats']
calling:  <class 'pyopia.instrument.silcam.SilCamLoad'>  with:  ['cl', 'filename', 'steps_string', 'timestamp', 'img', 'imc', 'imbw', 'stats']
calling:  <class 'pyopia.instrument.silcam.ImagePrep'>  with:  ['cl', 'filename', 'steps

In [6]:
# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

Modified:
    2023-01-08 23:03:57.220818
Pipeline steps:
    
1) Step: classifier
   Type: <class 'pyopia.classify.Classify'>
   Vars: {'model_path': 'keras_model.h5', 'class_labels': Index(['oil', 'other', 'bubble', 'faecal_pellets', 'copepod', 'diatom_chain',
       'oily_gas'],
      dtype='object'), 'model': <keras.engine.sequential.Sequential object at 0x0000016132598FD0>}
2) Step: load
   Type: <class 'pyopia.instrument.silcam.SilCamLoad'>
   Vars: {}
3) Step: imageprep
   Type: <class 'pyopia.instrument.silcam.ImagePrep'>
   Vars: {}
4) Step: segmentation
   Type: <class 'pyopia.process.Segment'>
   Vars: {'minimum_area': 12, 'threshold': 0.85, 'fill_holes': True}
5) Step: statextract
   Type: <class 'pyopia.process.CalculateStats'>
   Vars: {'max_coverage': 30, 'max_particles': 5000, 'export_outputpath': None, 'min_length': 0}
6) Step: output
   Type: <class 'pyopia.io.StatsH5'>
   Vars: {'datafilename': 'proc/test'}

PyOpia version:
    0.0.23


In [7]:
import pyopia.statistics
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))
print('Number of raw images: ', pyopia.statistics.count_images_in_stats(stats))

stats header:  Index(['major_axis_length', 'minor_axis_length', 'equivalent_diameter',
       'solidity', 'minr', 'minc', 'maxr', 'maxc', 'probability_oil',
       'probability_other', 'probability_bubble', 'probability_faecal_pellets',
       'probability_copepod', 'probability_diatom_chain',
       'probability_oily_gas', 'export name', 'timestamp', 'saturation'],
      dtype='object')
Total number of particles:  36904
Number of raw images:  4
